# Download and process WorldClim data into a set of GeoTIFFs at native resolution

In [1]:
import os
import sys
import shutil
import zipfile
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [12]:
# Get the required info from the config file
code_path            = cs.read_from_config(config_file,'code_path')
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
climate_path         = cs.read_from_config(config_file,'climate_path')
climate_url_file     = cs.read_from_config(config_file,'climate_urls')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Download data

In [7]:
download_folder = Path(data_path) / geospatial_temp_path / climate_path / 'download'

In [8]:
download_folder.mkdir(parents=True, exist_ok=True)

In [13]:
with open(Path(code_path)/climate_url_file) as url_file:
    for url in url_file:
        cs.download_url_into_folder(url,download_folder)

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tmin.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tmax.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tavg.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_prec.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_srad.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_wind.zip

Successfully downloaded https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_vapr.zip


### Unzip

In [14]:
files = cs.find_files_in_folder(download_folder, '.zip')

In [16]:
for file in files:
    with zipfile.ZipFile(download_folder/file, 'r') as zip_ref:
        zip_ref.extractall(download_folder)
    os.remove(download_folder/file)

### Subset files to domain

In [19]:
raw_folder = Path(data_path) / geospatial_temp_path / climate_path / 'raw'

In [20]:
raw_folder.mkdir(parents=True, exist_ok=True)

In [21]:
# Convert subsetting area into a usable GDAL setting
# subset_area = [lon_min, lon_max, lat_min, lat_max]
# GDAL window = [ulx, uly, lrx, lry]; [upper left x, upper left y, lower right x, lower right y]
# Mapping:
#   ulx = lon_min = subset_area[0]
#   uly = lat_max = subset_area[3]
#   lrx = lon_max = subset_area[1]
#   lry = lat_min = subset_area[2]
subset_coor = download_area.split(',')
window = [subset_coor[0], subset_coor[3], subset_coor[1], subset_coor[2]]

In [17]:
files = cs.find_files_in_folder(download_folder, '.tif')

In [25]:
for src_file in files:

    # Ensure we move the files into folders for each variable
    var = os.path.basename(src_file).split('_')[2] 
    des_folder = raw_folder / var
    des_folder.mkdir(parents=True, exist_ok=True)

    # Subset the file
    des_file = des_folder/os.path.basename(src_file)
    cs.subset_tif(src_file,des_file,window)

### Delete the downloads to save space

In [26]:
if os.path.isdir(download_folder): 
    shutil.rmtree(download_folder)